<a href="https://colab.research.google.com/github/valentinaaptr/ibm_capstone_escope/blob/main/project_escope_ibm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Melakukan install extension untuk akses Model AI
# ====================

!pip install langchain_community
!pip install replicate

In [ ]:
# Menyiapkan environment agar bisa menggunakan model IBM Granite melalui Replicate
# ====================
from langchain_community.llms import Replicate
import os
from google.colab import userdata
# Set the API token
api_token = userdata.get('api_token')
os.environ["REPLICATE_API_TOKEN"] = api_token
# Model setup
model = "ibm-granite/granite-3.3-8b-instruct"
output = Replicate(
 model=model,
 replicate_api_token=api_token,
)

In [ ]:
import pandas as pd

# Fungsi Perhitungan Rasio Keuangan
# =========================
def ratios(aset_lancar, liabilitas_lancar, total_liabilitas, total_ekuitas,
                 ebit, beban_bunga, harga_saham, jumlah_saham):
    der = round(total_liabilitas / total_ekuitas, 2) if total_ekuitas != 0 else None
    current_ratio = round(aset_lancar / liabilitas_lancar, 2) if liabilitas_lancar != 0 else None
    icr = round(ebit / beban_bunga, 2) if beban_bunga != 0 else None
    nilai_buku_per_saham = total_ekuitas / jumlah_saham if jumlah_saham != 0 else None
    pbv = round(harga_saham / nilai_buku_per_saham, 2) if nilai_buku_per_saham else None
    return der, current_ratio, icr, pbv

# Klasifikasi Tingkat Risiko Kredit
# =========================
def riskcredit_eval(icr, der, cr):
    count = 0
    if icr is not None and icr > 2:
        count += 1
    if der is not None and der < 1:
        count += 1
    if cr is not None and cr > 1:
        count += 1

    if count == 0 or count == 1:
        return "Rendah"
    elif count == 2:
        return "Sedang"
    else:
        return "Tinggi"

def get_float_input(prompt):
    while True:
        try:
            return float(input(prompt))
        except ValueError:
            print("Harap masukkan angka yang valid.")


# Melakukan Input Data
# =========================
data_input = []

while True:
    perusahaan = input("Kode Perusahaan: ").upper()

    while True:
        tahun = input("Tahun: ")
        if tahun.isdigit() and len(tahun) == 4:
            break
        else:
            print("Tahun harus berupa 4 digit angka. Contoh: 2025")

    opini = input("Opini Audit: ")

    print("\n--- Financial Ratio Indicators ---")

    aset_lancar = get_float_input("Aset Lancar: ")
    liabilitas_lancar = get_float_input("Liabilitas Lancar: ")
    total_liabilitas = get_float_input("Total Utang: ")
    total_ekuitas = get_float_input("Total Ekuitas: ")
    ebit = get_float_input("EBIT: ")
    beban_bunga = get_float_input("Beban Bunga: ")
    harga_saham = get_float_input("Harga Saham: ")
    jumlah_saham = get_float_input("Jumlah Saham Beredar: ")

    # Melakukan perhitungan rasio berdasarkan fungsi
    der, current_ratio, icr, pbv = ratios(aset_lancar, liabilitas_lancar,
                                                total_liabilitas, total_ekuitas,
                                                ebit, beban_bunga,
                                                harga_saham, jumlah_saham)

    tingkat_risiko = riskcredit_eval(icr, der, current_ratio)

# Memasukkan Data ke dalam List
# =========================
    data_input.append({
        'Perusahaan': perusahaan,
        'Tahun': tahun,
        'OpiniAudit': opini,
        'DER': der,
        'CurrentRatio': current_ratio,
        'ICR': icr,
        'PBV': pbv,
        'RisikoKredit': tingkat_risiko
    })

    lanjut = input("Apakah Anda ingin menambahkan data baru? (y/n): ")
    if lanjut.lower() != 'y':
        print("Input data selesai.")
        break


# Menampilkan Data dalam Tabel
# =========================
df = pd.DataFrame(data_input)

if not df.empty:
    df = df.sort_values(by=["Perusahaan", "Tahun"]).reset_index(drop=True)
    df.index += 1
    df.index.name = 'No'

    print("\n" + "=" * 80)
    print("📊 COMPANY PERFORMANCE SUMMARIES".center(80))
    print("=" * 80)
    print(df.to_string())

    # Opsi Menghapus Data
    # =========================
    hapus = input("\nApakah terdapat data yang ingin Anda hapus? (y/n): ")
    if hapus.lower() == 'y':
        tahun_hapus = input("Masukkan berdasarkan tahun: ")
        try:
            tahun_hapus = int(tahun_hapus)
            sebelum = len(df)
            df = df[df['Tahun'].astype(int) != tahun_hapus]
            sesudah = len(df)

            if sesudah < sebelum:
                print(f"✅ Data perusahaan untuk tahun {tahun_hapus} berhasil dihapus.\n")
            else:
                print(f"⚠️ Tidak ditemukan data dengan tahun {tahun_hapus}.")
            print(df.to_string())
        except ValueError:
            print("⚠️ Tahun yang Anda masukkan tidak valid.")

    # Opsi Menambah Data
    # =========================
    tambah = input("\nApakah Anda ingin menambahkan data baru? (y/n): ")
    if tambah.lower() == 'y':
        try:
            perusahaan = input("Masukkan nama/kode perusahaan: ").upper()
            tahun = int(input("Masukkan tahun (4 digit): "))
            opini = input("Masukkan opini audit: ")

            aset_lancar = get_float_input("Aset Lancar: ")
            liabilitas_lancar = get_float_input("Liabilitas Lancar: ")
            total_liabilitas = get_float_input("Total Utang: ")
            total_ekuitas = get_float_input("Total Ekuitas: ")
            ebit = get_float_input("EBIT: ")
            beban_bunga = get_float_input("Beban Bunga: ")
            harga_saham = get_float_input("Harga Saham: ")
            jumlah_saham = get_float_input("Jumlah Saham Beredar: ")

            der, current_ratio, icr, pbv = ratios(aset_lancar, liabilitas_lancar,
                                                        total_liabilitas, total_ekuitas,
                                                        ebit, beban_bunga,
                                                        harga_saham, jumlah_saham)

            tingkat_risiko = riskcredit_eval(icr, der, current_ratio)

            new_data = {
                'Perusahaan': perusahaan,
                'Tahun': tahun,
                'OpiniAudit': opini,
                'DER': der,
                'CurrentRatio': current_ratio,
                'ICR': icr,
                'PBV': pbv,
                'RisikoKredit': tingkat_risiko
            }

            # Menampilkan Data Baru dalam Tabel
            # =========================
            df = pd.concat([df.reset_index(drop=True), pd.DataFrame([new_data])], ignore_index=True)
            df.index += 1
            df.index.name = 'No'
            print("\n✅ Data berhasil ditambahkan.")
            print(df.to_string())

        except Exception as e:
            print(f"⚠️ Gagal menambahkan data: {e}")
else:
    print("❌ Data kosong. Tidak ada data untuk ditampilkan.")


Kode Perusahaan: kren
Tahun: 2020
Opini Audit: Menurut opini kami, laporan keuangan konsolidasian  terlampir menyajikan secara wajar, dalam semua hal  yang material, posisi keuangan konsolidasian  PT Kresna Graha Investama Tbk dan entitas anaknya tanggal 31 Desember 2021, serta kinerja keuangan,  dan arus kas konsolidasiannya untuk tahun yang  berakhir pada tanggal tersebut, sesuai dengan standar  akuntansi keuangan di Indonesia

--- Financial Ratio Indicators ---
Aset Lancar: 2201259708255
Liabilitas Lancar: 417991713481
Total Utang: 585507980261
Total Ekuitas: 2745296758850
EBIT: -215526941851
Beban Bunga: 22516501266
Harga Saham: 84
Jumlah Saham Beredar: 18208470100
Apakah Anda ingin menambahkan data baru? (y/n): n
Input data selesai.

                        📊 COMPANY PERFORMANCE SUMMARIES                         
   Perusahaan Tahun                                                                                                                                                       

In [97]:
# Mengaktifkan word wrap untuk output teks panjang di Colab
from IPython.core.display import display, HTML

display(HTML('''
<style>
  pre {
    white-space: pre-wrap !important;  /* Wrap text */
    word-break: break-word !important; /* Prevent long words from overflowing */
  }
  .output_result {
    max-height: none !important; /* Remove scroll */
  }
  .output_area {
    overflow-y: visible !important; /* Prevent vertical scrolling */
  }
</style>
'''))

  # Penggabungan semua baris data jadi satu string teks
  # =========================
summaries = ""
for idx, row in df.iterrows():
      summaries += f"""
  Perusahaan: {row['Perusahaan']}
  Tahun: {row['Tahun']}
  Opini Audit: {row['OpiniAudit']}
  DER: {row['DER']}
  Current Ratio: {row['CurrentRatio']}
  ICR: {row['ICR']}
  PBV: {row['PBV']}
  Risiko Kredit: {row['RisikoKredit']}
  ----------------------------
  """

  # Membuat refined prompt untuk semua data
  # =========================
refined_prompt = f"""
  Berikut adalah data ringkasan laporan audit dan keuangan dari beberapa perusahaan:

  {summaries}

  Berdasarkan data laporan audit dan keuangan perusahaan, buatlah analisis naratif yang mencakup jenis opini audit (WTP, WDP, TMP, atau Disclaimer), indikasi adanya masalah going concern, penilaian risiko kredit berdasarkan DER, Current Ratio, dan ICR, serta evaluasi kinerja keuangan yang dikaitkan dengan nilai PBV (misalnya undervalued atau overvalued). Sertakan juga rekomendasi untuk pemangku kepentingan dan pemegang saham. Tulis analisis ini dalam satu paragraf berbahasa Indonesia yang padat dan profesional. Jangan gunakan format daftar atau penomoran. Cukup satu paragraf saja.

  Berikut output narasi yang diharapkan:
  Opini audit yang disampaikan merupakan Wajar Tanpa Pengecualian (WTP), namun rasio keuangan menunjukkan adanya risiko going concern yang signifikan. Hal ini terlihat dari ICR yang rendah (0.5) dan Current Ratio di bawah 1, menunjukkan keterbatasan kemampuan perusahaan dalam memenuhi kewajiban jangka pendek dan bunga pinjaman. DER yang tinggi (2.8) semakin memperkuat indikasi tekanan struktur modal, sehingga risiko kredit dikategorikan tinggi. Kinerja keuangan secara umum menunjukkan ketergantungan pada utang, dan PBV sebesar 0.6 mencerminkan penilaian pasar yang rendah terhadap nilai perusahaan, kemungkinan karena kekhawatiran terhadap keberlangsungan usaha dan efisiensi pengelolaan modal. Secara keseluruhan, meskipun laporan disusun secara wajar, terdapat tanda-tanda kesehatan keuangan yang perlu perhatian serius.
  """

  # Pengaturan Parameters Output
  # =========================
parameters = {
    "top_k": 5,
    "top_p": 1.0,
    "temperature": 0.5,
    "repetition_penalty": 1.1,
    "max_new_tokens": 1000,
    "min_tokens": 500,
    "stop_sequence": "NONE",
    "random_seed": None
}

response = output.invoke(refined_prompt, parameters)

  # Print Output Hasil Analisis
  # =========================
print("📰COMPANY EVALUATION SUMMARIES:\n")
print(response)


📰COMPANY EVALUATION SUMMARIES:

Opini audit yang diberikan oleh auditor untuk PT Kresna Graha Investama Tbk (KREN) untuk tahun 2020 dan 2021 adalah Wajar Tanpa Pengecualian (WTP), yang menunjukkan bahwa laporan keuangan konsolidasinya secara keseluruhan menyajikan secara wajar dan benar. Namun, analisis rasio keuangan menciptakan indikasi risiko going concern yang signifikan. ICR yang negatif (-9.57 untuk 2021 dan -14.85 untuk 2021) dan Current Ratio di bawah 1 pada tahun 2021 menunjukkan keterbatasan kemampuan perusahaan dalam memenuhi kewajiban jangka pendek dan bunga pinjaman, sehingga mencerminkan potensi masalah liquidity. DER yang tinggi (0.21 untuk 2020 dan 0.26 untuk 2021) menunjukkan struktur modal yang lebih banyak berbentuk utang, yang bisa meningkatkan risiko kredit. PBV yang rendah (0.56 untuk 2020 dan 0.7 untuk 2021) mungkin indikasi bahwa pasar memandang perusahaan sebagai undervalued, tetapi ini juga bisa dibaca sebagai bagian dari kekhawatiran terhadap keberlangsungan 